In [1]:
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
def create_scene():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_model:
    name: pool_ball
    file: file://{ball_sdf_file}
- add_model:
    name: pool_ball2
    file: file://{ball_sdf_file}
    default_free_body_pose:
        ball:
            translation: [-0.5, 0.0, 0.5]
- add_weld:
    parent: world
    child: table_top::table_top_center   
"""
    
    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


# create_scene()

In [4]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>1.5 1.0 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>1.5 1.0 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>
"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

In [5]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (ball.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
ball_sdf_file = os.path.join(temp_dir, "ball.sdf")
ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="ball">
    <link name="ball">
      <inertial>
        <mass>0.5</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="visual">
        <pose>0.0 0.0 0.05 0 0 0</pose>
        <geometry>
            <sphere>
                <radius>0.05</radius>
            </sphere>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="visual">
        <pose>0.0 0.0 0.05 0 0 0</pose>
        <geometry>
            <sphere>
                <radius>0.05</radius>
            </sphere>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

with open(ball_sdf_file, "w") as f:
    f.write(ball_sdf)

ball_sdf_file2 = os.path.join(temp_dir, "ball2.sdf")
ball_sdf2 = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="ball2">
    <link name="ball2">
      <inertial>
        <mass>0.5</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="visual">
        <pose>0.3 0.0 0.05 0 0 0</pose>
        <geometry>
            <sphere>
                <radius>0.05</radius>
            </sphere>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="visual">
        <pose>0.3 0.0 0.05 0 0 0</pose>
        <geometry>
            <sphere>
                <radius>0.05</radius>
            </sphere>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

with open(ball_sdf_file2, "w") as f:
    f.write(ball_sdf2)

In [6]:
from manipulation import ConfigureParser
from pydrake.all import (
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    MeshcatVisualizer,
    StartMeshcat,
    MeshcatVisualizerParams,
    Simulator,
    JointSliders,
    InverseKinematics,
    RotationMatrix,
    Solve,
    ContactVisualizerParams,
    ContactVisualizer,
    GeometrySet,
    CollisionFilterDeclaration,
    Role,
    eq,
    RigidTransform,
)

def build_env():
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant)
    ConfigureParser(parser)

    # Load the SDF files for the table and sphere.
    parser = Parser(plant)
    parser.AddModels(table_top_sdf_file) 
    parser.AddModels(ball_sdf_file) 
    parser.AddModels(ball_sdf_file2) 
    print('models loaded.')

    # Find the table_top_center frame and world frame.
    table_top_center_frame = plant.GetFrameByName("table_top_center")
    world_frame = plant.world_frame()

    # Add a weld between the table_top_center frame and the world frame
    table_instance = plant.GetModelInstanceByName("table_top")  # Replace with the actual model instance name
    table_instance_frame = plant.GetFrameByName("table_top_center", table_instance)
    plant.WeldFrames(world_frame, table_top_center_frame)
    print('weld added.')
    
    plant.Finalize()

    meshcat.Delete()
    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

In [7]:
import numpy as np
import pydrake.all
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, Simulator, 
)
from pydrake.multibody.parsing import Parser

diagram, plant, scene_graph = build_env()

context = diagram.CreateDefaultContext()
# plant_context = plant.GetMyContextFromRoot(context)
plant_context = diagram.GetMutableSubsystemContext(plant, context)

# Set the initial velocity for the pool ball.
pool_ball_model = plant.GetModelInstanceByName("ball")  # Replace with the actual model instance name
pool_ball_model2 = plant.GetModelInstanceByName("ball2")  # Replace with the actual model instance name
# initial_positions = np.array([0.0, 0.0, -1.0, 0.0, 1.0, 0.0, 0.0])
initial_velocity = np.array([0.0, 0.0, 0.0, 2.0, 0.0, 0.0])

# plant.SetPositions(plant_context, pool_ball_model, initial_positions)
plant.SetVelocities(context=plant_context, model_instance=pool_ball_model2, v=np.zeros(6))
plant.SetVelocities(context=plant_context, model_instance=pool_ball_model, v=initial_velocity)

# Create the simulator and set the initial conditions.
simulator = Simulator(diagram, context)

# Simulate for a desired duration.
meshcat.StartRecording(set_visualizations_while_recording=False)
duration = 10
simulator.AdvanceTo(duration)
meshcat.PublishRecording()


models loaded.
weld added.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=45359394-3853-4f02-980c-9c91afed0646' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>